## 特征工程
### 使用找空缺星期天以及空缺节假日的方法将真实日期还原出来。
### 节假日之前的around_holiday呈线性递增趋势，节假日之后的around_holiday呈线性递减趋势。
### 使用真实日期在数据中提取一些特征，例如该天是否放假，是否是星期天等等。
### 最终生成每个brand的dataframe存为csv。

In [3]:
import pandas as pd


# 上班的星期天
def spring_sunday(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 6 and all_data_fill[pd.to_datetime(all_data_fill['date']) == date]['is_work'].values[0] == 1:
        return 1
    else:
        return 0


# 正常的Sunday
def is_norm_sunday(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 6 and all_data_fill[pd.to_datetime(all_data_fill['date']) == date]['is_work'].values[0] == 0:
        return 1
    else:
        return 0


def is_redundent_sunday(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 6 and all_data_fill[pd.to_datetime(all_data_fill['date']) == date]['date_old'].values[0] == 0:
        return 1
    else:
        return 0


# 上班的saturday
def is_saturday(ds):
    date = pd.to_datetime(ds)
    if date.weekday() == 5 and all_data_fill[pd.to_datetime(all_data_fill['date']) == date]['is_work'].values[0] == 1:
        return 1
    else:
        return 0


def is_in_holiday(ds):
    date = pd.to_datetime(ds)
    if all_data_fill[pd.to_datetime(all_data_fill['date']) == date]['is_holiday'].values[0] == 1:
        return 1
    else:
        return 0


def is_after_new_year(ds):
    date = pd.to_datetime(ds)
    if date.month == 1 and date.day <= 8:
        return 1
    else:
        return 0


def get_month(ds):
    date = pd.to_datetime(ds)
    return date.month

def get_year(ds):
    date = pd.to_datetime(ds)
    return date.year

def get_day(ds):
    date = pd.to_datetime(da)
    return date.day


## 生成趋势
1. 通过之前A榜的train_data对2016年4月7号之后（没有使用答案）的总上牌量做预测，可以大概得出之后的发展趋势。
2. 试过挑选B榜中的几个牌子的总和来做，但效果没使用A榜的预测好。
3. 对所以上牌量数据进行log平滑处理。
4. 在training set中进行validation，选取lightgbm的参数

In [1]:
import pandas as pd
import numpy as np
import lightgbm
from sklearn.metrics import mean_squared_error

# train = pd.read_table("../data/fusai_train_20180227.txt", engine='python')
# train = train[["date", "brand", "cnt"]]


# train = train[train['brand'].isin([2, 6, 4, 10])]

# group_day_t = train.groupby(['date'], as_index=False)['cnt'].agg({'count1': np.sum})
d_data = pd.read_csv('../data/train_brand_1/train_brand_1.csv')

all_length = 1800
split_point = 1191
cv_split = 900
# all_data = pd.merge(d_data, group_day_t, on='date', how='left')
all_data = d_data
all_data = all_data.fillna(0)[:all_length]

feature_data = all_data[['week_no', 'day_of_week', 'date', 'lunar_day', 'is_holiday', 'after_holiday_type',
                         'after_holiday', 'before_holiday', 'before_holiday_type', 'before_holiday',
                         'last_workday_before_holiday', 'is_week_end', 'is_work', 'spring_sunday', 'is_norm_sunday',
                         'is_saturday', 'is_after_new_year', 'month', 'day', 'year']]
feature_data.index = range(len(feature_data))

value = list(all_data['cnt_y'].values)
ds = list(all_data['ds'].values)

train_data = feature_data[:split_point]
test_data = feature_data[split_point:all_length]

train_value = value[:split_point]
train_log = np.log(list(map(lambda x: x + 1, train_value)))

test_value = value[cv_split:split_point]


lgbm = lightgbm.LGBMRegressor(objective='regression', learning_rate=0.13, max_depth=6,
                              n_estimators=200, n_jobs=8, subsample=0.8)
lgbm.fit(train_data, train_log, verbose=False)
predictions = lgbm.predict(test_data)
predictions = list(map(lambda x: x - 1, np.exp(predictions)))

test_data.index = range(len(test_data))
test_data['test'] = predictions
test_data['ds'] = ds[split_point: all_length]
train_data['test'] = train_value
train_data['ds'] = ds[:split_point]

td = pd.concat([train_data, test_data], axis=0)
tdd = td[['ds', 'test']]
tdd.index = range(len(tdd))
tdd.loc[tdd['test'] < 0, 'test'] = 0
tdd.to_csv("../data/td.csv")

"""
0.13 6 0.8
"""

# lr = np.arange(0.01, 0.21, 0.01)
# md = np.arange(1, 11, 1)
# ss = np.arange(0.1, 1.01, 0.1)
# mse_min = 10000000
# tp = ()
# for l in lr:
#     for m in md:
#         for s in ss:
#             lgbm = lightgbm.LGBMRegressor(objective='regression', learning_rate=l, max_depth=m,
#                                           n_estimators=200, n_jobs=8, subsample=s)
#             lgbm.fit(train_data, train_log, verbose=False)
#             predictions = lgbm.predict(test_data)
#             mse = mean_squared_error(test_value, list(map(lambda x: x - 1, np.exp(predictions))))
#
#             if mse_min > mse:
#                 mse_min = mse
#                 tp = (l, m, s)
# print(mse_min)
# print(tp)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

'\n0.13 6 0.8\n'

## 分别对每个brand进行建模
1. 分别读每个brand的数据，与之前预测出来的未来趋势进行join，使趋势称为其中一个feature。
2. 使用ligtgbm进行回归建模，试过xgboost，效果不如lightgbm。
3. 最终的结果使用json格式数据存储

In [2]:
import pandas as pd
import lightgbm
from sklearn.metrics import mean_squared_error
import xgboost
import numpy as np
import json

"""
    1: 0.03, 2, 0.8 13431.09 0.15 1, 0.8 13594
    2: 0.05, 2, 0.9 10070.78 9493 0.1, 9, 0.5  (0.18 3 0.5 9492 log)
    3: 0.02, 7, 0.6 10255.07 10251 0.17, 8, 0.3 (0.17 8 0.3 12031 log)
    4: 0.05, 5, 1 11262.57 10018 0.16 3 0.7 (0.16 3 0.7 10022)
    5: 0.03, 3, 0.9 32225.13 30818 0.18 8 0.7 (0.18 8 0.7 30815)
    6: 0.06, 3, 1 4145.77 3357 0.2 4 0.6  (0.2 4 0.6 3366)
    7: 0.08, 2, 0.9 12853.56 12103 0.2 1 0.7 (0.2 1 0.7 12103)
    8: 0.08, 5, 0.6 76385.08 56536 0.17 7 0.4 (0.17 7 0.4 56528)
    9: 0.08, 10, 1 193139 169557 0.19 5 0.2 (0.19 5 0.2 169551)
    10: 0.08, 10, 1 24486 11048 0.01 7 0.6 (0.01 7 0.6 11046)
"""
# dict1 = {}
# for i in range(1, 11):
#     # b_data = pd.read_csv('../data/train_brand_1/b.csv')
#     # b_data.loc[pd.isna(b_data['date']), 'cnt'] = 0
#     td = pd.read_csv('../data/td.csv')
#     all_data = pd.read_csv('../data/train_brand_8/train_brand_' + str(i) + '.csv')
#     all_data = pd.merge(all_data, td, on='ds', how='left')
#     #all_data[np.isnan(all_data['cnt_x'])]['brand_fill'] = 0
#     all_data.loc[pd.isna(all_data['cnt_x']), 'brand_fill'] = 0
#
#     validate_point = 1050
#     split_point = 1379
#     all_length = 1800
#     start = 0
#     all_data = all_data[start:]
#     all_data = all_data.fillna(0)[:all_length]
#     all_data.index = range(len(all_data))
#     value = list(all_data['test'].values)
#     vt_value = value[validate_point: split_point]
#     cnty = list(map(lambda x: x + 1, all_data['test'].values))
#     all_data['test'] = np.log(cnty)
#
#     value = np.log(list(map(lambda x: x + 1, value)))
#
#     data_feature = all_data[['week_no', 'day_of_week', 'lunar_day', 'is_holiday', 'after_holiday_type',
#                              'after_holiday', 'before_holiday_type', 'before_holiday', 'last_workday_before_holiday',
#                              'is_week_end', 'is_work', 'spring_sunday', 'test', 'is_norm_sunday', 'is_saturday',
#                              'is_after_new_year', 'month', 'day', 'year', 'date', 'brand_fill']]
#
#     validate_data = data_feature[:validate_point]
#     validate_value = value[:validate_point]
#
#     vt_data = data_feature[validate_point: split_point]
#
#
#
#
#     lr = np.arange(0.1, 0.31, 0.01)
#     md = np.arange(2, 11, 1)
#     ss = np.arange(0.1, 1.01, 0.1)
#     mse_min = 1000000
#     tp = ()
#     for l in lr:
#         for m in md:
#             for s in ss:
#                 lgbm = lightgbm.LGBMRegressor(objective='regression', learning_rate=l, max_depth=m,
#                                               n_estimators=200, n_jobs=8, subsample=s)
#                 lgbm.fit(validate_data, validate_value, verbose=False)
#                 predictions = lgbm.predict(vt_data)
#                 mse = mean_squared_error(vt_value, list(map(lambda x: x - 1, np.exp(predictions))))
#
#                 if mse_min > mse:
#                     mse_min = mse
#                     tp = (l, m, s)
#     print(mse_min)
#     print(tp)
#     dict1[i] = str(tp[0]) + " " + str(tp[1]) + " " + str(tp[2]) + " " + str(mse_min)
#
# for k, v in dict1.items():
#     print(k)
#     print(v)
#
# with open('../data/knn_maps.json', 'w') as f:
#     json.dump(dict1, f)

'\n    1: 0.03, 2, 0.8 13431.09 0.15 1, 0.8 13594\n    2: 0.05, 2, 0.9 10070.78 9493 0.1, 9, 0.5  (0.18 3 0.5 9492 log)\n    3: 0.02, 7, 0.6 10255.07 10251 0.17, 8, 0.3 (0.17 8 0.3 12031 log)\n    4: 0.05, 5, 1 11262.57 10018 0.16 3 0.7 (0.16 3 0.7 10022)\n    5: 0.03, 3, 0.9 32225.13 30818 0.18 8 0.7 (0.18 8 0.7 30815)\n    6: 0.06, 3, 1 4145.77 3357 0.2 4 0.6  (0.2 4 0.6 3366)\n    7: 0.08, 2, 0.9 12853.56 12103 0.2 1 0.7 (0.2 1 0.7 12103)\n    8: 0.08, 5, 0.6 76385.08 56536 0.17 7 0.4 (0.17 7 0.4 56528)\n    9: 0.08, 10, 1 193139 169557 0.19 5 0.2 (0.19 5 0.2 169551)\n    10: 0.08, 10, 1 24486 11048 0.01 7 0.6 (0.01 7 0.6 11046)\n'

## 生成数据
生成每个牌子每天的数据，存到data/tb/ 中。

In [3]:
dict1 = {1: [0.19, 3, 0.8],
         2: [0.17, 6, 0.8],
         3: [0.19, 9, 0.8],
         4: [0.19, 9, 0.8],
         5: [0.17, 10, 0.8],
         6: [0.15, 10, 0.8],
         7: [0.19, 5, 0.8],
         8: [0.21, 4, 0.8],
         9: [0.16, 9, 0.8],
         10: [0.19, 9, 0.8]}


split_point = 1191
all_length = 1800
start = 0

for key, val in dict1.items():
    td = pd.read_csv('../data/td.csv')
    all_data = pd.read_csv('../data/train_brand_1/train_brand_' + str(key) + '.csv')
    all_data = pd.merge(all_data, td, on='ds', how='left')
    all_data.loc[pd.isna(all_data['cnt_x']), 'brand_fill'] = 0
    all_data = all_data[start:]

    all_data = all_data.fillna(0)[:all_length]
    all_data.index = range(len(all_data))

    # cnty = all_data['cnt_y'].values
    cnty = list(map(lambda x: x + 1, all_data['test'].values))
    all_data['test'] = np.log(cnty)

    value = list(all_data['cnt_x'].values)

    value = np.log(list(map(lambda x: x + 1, value)))


    data_feature = all_data[['week_no', 'day_of_week', 'lunar_day', 'is_holiday', 'after_holiday_type',
                             'after_holiday', 'before_holiday_type', 'before_holiday', 'last_workday_before_holiday',
                             'is_week_end', 'is_work', 'spring_sunday', 'test', 'is_norm_sunday', 'is_saturday',
                             'is_after_new_year', 'month', 'day', 'year', 'date', 'brand_fill']]

    train_data = data_feature[:split_point]
    train_label = value[:split_point]


    test_data = data_feature[split_point:all_length]

    lgbm = lightgbm.LGBMRegressor(objective='regression', learning_rate=val[0], max_depth=val[1],
                                  n_estimators=200, n_jobs=8, subsample=val[2])
    lgbm.fit(train_data, train_label, verbose=False)
    predictions = lgbm.predict(test_data)
    predictions = list(map(lambda x: x - 1, np.exp(predictions)))

    test_data.index = range(len(test_data))
    p_value = pd.DataFrame(predictions, columns=['cnt'])
    test_data['cnt'] = p_value['cnt']
    test_data = test_data[test_data['date'] != 0]

    test_data.loc[test_data['cnt'] < 0, 'cnt'] = 20

    test_data = test_data[['date', 'cnt', 'brand_fill']]
    test_data.to_csv("../data/tb2/" + str(key) + ".csv")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

## 数据整合
将各brand的数据整合在一个文件里进行提交。

In [4]:
import pandas as pd
import os


length = 600
file_dict = {}
for f in os.listdir("../data/tb2"):
    file_dict[f] = pd.read_csv("../data/tb2/" + f)

file_dict = dict(sorted(file_dict.items(), key=lambda x: int(x[0].split(".")[0])))
# print(file_dict.keys())

table = pd.DataFrame()
first = True
for key, value in file_dict.items():
    if first:
        table["date"] = file_dict[key]["date"]
    k = key.split(".")[0]
    table[k] = file_dict[key]["cnt"]
    table[k + "fill"] = file_dict[key]["brand_fill"]
table.describe()

brand = list(range(1, 11))

file_ob = open('../data/sample3.txt', 'w+')
for i in range(len(table)):
    row = table.iloc[i]
    date = row["date"]
    for b in brand:
        if row[str(b) + "fill"] != 0:
            string = str(int(date)) + "\t" + str(b) + "\t" + str(int(row[str(b)])) + "\n"
            file_ob.write(string)